In [1]:
from core.llm_client import LLMClient
import os

model_type = os.getenv("MODEL_TYPE", "openai")
llm_client = LLMClient(model_type=model_type)

In [4]:
from prompts.discovery import SYSTEM_PROMPT
from tools.registry import ToolRegistry
from utility import load_json

registry_json = load_json(os.path.join("tools", "registry.json"))

In [5]:
user_prompt = {
    "id": 1,
    "type": "straightforward",
    "conversation_context": [
      {"role": "user", "content": "Show me Apple laptops under $1200 with at least 16GB RAM"}
    ],
    "current_query": "Show me Apple laptops under $1200 with at least 16GB RAM",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": registry_json,
    "spec_keys": ["brand", "ram_gb", "storage_gb", "screen_size_inches"]
  }

In [7]:
user_prompt

{'id': 1,
 'type': 'straightforward',
 'conversation_context': [{'role': 'user',
   'content': 'Show me Apple laptops under $1200 with at least 16GB RAM'}],
 'current_query': 'Show me Apple laptops under $1200 with at least 16GB RAM',
 'slots_till_now': {},
 'fsm_state': 'COLLECTING',
 'available_tools': {'DISCOVERY': {'available_tools': [{'name': 'filter_products',
     'description': 'Filter products in a subcategory using specifications like RAM, storage, screen size, etc.',
     'mandatory_params': [{'name': 'subcategory',
       'type': 'string',
       'description': 'subcategory of the product.',
       'examples': ['laptop', 'mobile', 'treadmill']},
      {'name': 'category',
       'type': 'string',
       'description': 'category of the product.',
       'examples': ['electronics', 'sports']}],
     'optional_params': [{'name': 'brand',
       'type': 'string',
       'description': 'subcategory of the product.',
       'examples': ['Apple', 'lenovo', 'samsung']},
      {'nam

### Input-1